In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.cross_validation import train_test_split
import numpy as np
import pandas as pd

/Users/YorkLin/.pyenv/versions/3.5.2/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
cancer = load_breast_cancer()

In [3]:
x_train,x_test,y_train,y_test = train_test_split(cancer.data,cancer.target,test_size=0.2)

In [47]:
pd.DataFrame(x_train).shape

(455, 30)

In [48]:
pd.DataFrame(y_train).shape

(455, 1)

In [49]:
pd.DataFrame(x_test).shape

(114, 30)

In [10]:
from sklearn.model_selection import StratifiedKFold
StratifiedKFold(n_splits=10,random_state=1)

StratifiedKFold(n_splits=10, random_state=1, shuffle=False)

In [11]:
from sklearn.model_selection import StratifiedKFold
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((0,1),float)
    train_pred=np.empty((0,1),float)
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        test_pred=np.append(test_pred,model.predict(test))
    return test_pred.reshape(-1,1),train_pred

In [15]:
from sklearn import tree
model1 = tree.DecisionTreeClassifier(random_state=1)
test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=pd.DataFrame(x_train),test=pd.DataFrame(x_test),y=pd.Series(y_train))
train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

In [17]:
test_pred1

,0
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
5,0.0
6,1.0
7,1.0
8,1.0
9,1.0


In [35]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier()
test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=pd.DataFrame(x_train),test=pd.DataFrame(x_test),y=pd.Series(y_train))
train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

In [36]:
voted_test_pred1 = pd.DataFrame(list(map(lambda x: 1 if x>=5 else 0,test_pred1[0].values.reshape(10,y_test.shape[0]).sum(axis=0))))
voted_test_pred2 = pd.DataFrame(list(map(lambda x: 1 if x>=5 else 0,test_pred2[0].values.reshape(10,y_test.shape[0]).sum(axis=0))))

In [43]:
from sklearn.linear_model import LogisticRegression
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([voted_test_pred1, voted_test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test, y_test)

0.9122807017543859

In [253]:
model1.score(x_test,y_test),model2.score(x_test,y_test)

(0.9385964912280702, 0.9210526315789473)